In [14]:
import numpy as np
import random
import matplotlib.pyplot as plt
import subprocess

In [17]:
brew install cmake python glew freeglut

SyntaxError: invalid syntax (3342952221.py, line 1)

In [7]:
from pyrosetta import init
init()
print("PyRosetta initialized successfully!")


ModuleNotFoundError: No module named 'pyrosetta'

In [10]:
conda install numpy matplotlib jupyter -y

Retrieving notices: ...working... done
Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - jupyter
    - matplotlib
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.11.26 |       hca03da5_0         132 KB
    certifi-2024.12.14         |  py311hca03da5_0         163 KB
    conda-24.11.1              |  py311hca03da5_0         1.2 MB
    matplotlib-3.9.2           |  py311hca03da5_1           8 KB
    matplotlib-base-3.9.2      |  py311h7ef442a_1         7.9 MB
    ------------------------------------------------------------
                                           Total:         9.4 MB

The following packages will be UPDATED:

  ca-certificates                      2024.9.24-hca03da5_0 --> 2024.11.26-hca03da5_0 
  certifi                         

In [12]:
ls /path/to/pyrosetta.whl


ls: /path/to/pyrosetta.whl: No such file or directory


In [13]:
pwd

'/Users/apple/Desktop/NIC/Reflective Journal'